In [1]:
# STARTER CODE FOR STIGLER'S DIET PROBLEM
using NamedArrays
using CSV

# import Stigler's data set
raw = CSV.read("stigler.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );


MethodError: MethodError: no method matching NamedArray(::DataFrames.DataFrame, ::Tuple{Array{Union{Missing, String},1},DataFrames.DataFrameRow{DataFrames.DataFrame,DataFrames.SubIndex{DataFrames.Index,UnitRange{Int64},UnitRange{Int64}}}}, ::Tuple{String,String})
Closest candidates are:
  NamedArray(!Matched::AbstractArray{T,N}, !Matched::Tuple{}, ::Tuple{Vararg{Any,N}}) where {T, N} at /home/laibly/.julia/packages/NamedArrays/uMdmN/src/constructors.jl:29
  NamedArray(!Matched::AbstractArray{T,N}, !Matched::Tuple{Vararg{OrderedCollections.OrderedDict,N}}, ::Tuple{Vararg{Any,N}}) where {S<:NamedArray, T, N} at /home/laibly/.julia/packages/NamedArrays/uMdmN/src/namedarraytypes.jl:49
  NamedArray(!Matched::AbstractArray{T,N}, !Matched::Tuple{Vararg{Array{T,1} where T,N}}, ::Tuple{Vararg{Any,N}}) where {T, N} at /home/laibly/.julia/packages/NamedArrays/uMdmN/src/constructors.jl:48

In [2]:
using JuMP, Clp

m = Model(solver=ClpSolver())

#The amount of money spent on a given food item
@variable(m, moneyForGivenFoodItem[1:size(n_foods,1)]>=0)

#this matrix is similar to data expect each row is multiple by amount of money spent on
#the food for the corresponding row
@variable(m, foodCost[1:size(n_foods,1), 1:size(n_nutrients,1)])


for i in 1:size(n_foods,1)
    @constraint(m, foodCost[i, :] .== data[i,:]*moneyForGivenFoodItem[i])
end

#Every nutrient needs to be greater than or equal to the RDA
for j = 1:size(n_nutrients,1)
    @constraint(m, sum(foodCost[:,j]) >= lower[nutrients[j]])
end

#want to minimize the amount of money spent on food
@objective(m, Min, sum(moneyForGivenFoodItem[:]))

#solve
solve(m)
println("Total Annual Cost: ", getobjectivevalue(m)*365)
for i in 1:77
    
    if(getvalue(moneyForGivenFoodItem[i])>0)  
    count = (getvalue(moneyForGivenFoodItem[i])*data[i, :]) + count
    println("Food that make up diet: "foods[i])
    end
end

UndefVarError: UndefVarError: data not defined

In [3]:
using JuMP, Clp

m = Model(solver=ClpSolver())

#The amount of money spent on a given food item
@variable(m, moneyForGivenFoodItem[1:size(n_foods,1)]>=0)

#this matrix is similar to data expect each row is multiple by amount of money spent on
#the food for the corresponding row
@variable(m, foodCost[1:size(n_foods,1), 1:size(n_nutrients,1)])


for i in 1:size(n_foods,1)
    @constraint(m, foodCost[i, :] .== data[i,:]*moneyForGivenFoodItem[i])
end

#Every nutrient needs to be greater than or equal to the RDA
for j = 1:size(n_nutrients,1)
    @constraint(m, sum(foodCost[:,j]) >= lower[nutrients[j]])
end

#Convert to vegetarian
@constraint(m, foodCost[30,:] .== 0)

#want to minimize the amount of money spent on food
@objective(m, Min, sum(moneyForGivenFoodItem[:]))

#solve
solve(m)
println("Total Cost: ", getobjectivevalue(m)*365)
for i in 1:77
    
    if(getvalue(moneyForGivenFoodItem[i])>0)  
    count = (getvalue(moneyForGivenFoodItem[i])*data[i, :]) + count
    println("Food that make up diet: "foods[i])
    end
end

UndefVarError: UndefVarError: data not defined